In [ ]:
import pathlib
import cv2
import matplotlib.pyplot as plt
import numpy as np

import facerecognition
import clustering

In [ ]:
def img_show(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)

In [ ]:
root_dir = pathlib.Path().resolve()
features_dir = root_dir.joinpath("features")

if not features_dir.exists():
    features_dir.mkdir()


In [ ]:
images_dir = root_dir.joinpath("images")
print(images_dir)
print(images_dir.exists())

images_path = images_dir.glob('**/*.jpg')

print(images_path)


In [ ]:
face = facerecognition.Face()

In [ ]:
for img_path in images_path:
    face.img2features(img_path, features_dir)

## Clustering

In [ ]:
feature_list = np.zeros(128)  # dummy

npz_files = features_dir.glob('**/*.npz')
npz_files = list(npz_files)

for npz_path in npz_files:
    img, feature = face.load_img_feature(npz_path)
    feature_list = np.vstack((feature_list, feature))

feature_list = feature_list[1:]
print(feature_list.shape)

In [ ]:
nclusters = 4  # nclusters

kmeans = clustering.Kmeans()
compactness, labels, centers = kmeans.apply(feature_list, nclusters)
print(labels.ravel())

In [ ]:
for npz_path, label in zip(npz_files, labels.ravel()):
    rename = str(label) + "_" + npz_path.name
    rename_path = npz_path.parent.joinpath(rename)
    npz_path.rename(rename_path)

## Verification

In [ ]:
def img_blank(n, height=112, width=112):
    if n==0:
        height = 0
    blank = np.zeros((height, width*n, 3))
    blank += 255  # white
    return np.uint8(blank)

In [ ]:
npz_files = features_dir.glob('**/*.npz')

img_list = [[] for _ in range(nclusters)]
for npz_path in npz_files:
    i = npz_path.name.find("_")
    label = int(npz_path.name[:i])  # <class 'int'>

    img, feature = face.load_img_feature(npz_path)
    img_list[label].append(img)

# Number of clusters in each
num_nclusters = [ len(img_list[i]) for i in range(nclusters) ]
print(num_nclusters)

In [ ]:
max_num = max(num_nclusters)
for label, num in enumerate(num_nclusters):
    n_blank = max_num - num
    if n_blank > 0:
        blank = img_blank(n_blank)
        img_list[label].append(blank)

In [ ]:
img_row = [ cv2.hconcat([*img_list[i]]) for i in range(nclusters) ]
img_tile = cv2.vconcat([*img_row])
img_show(img_tile)

In [ ]:
img_save_path = root_dir.joinpath("tile.jpg")

cv2.imwrite(str(img_save_path), img_tile)